<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_11_3_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 11: Finetuning**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 11 Material

Module 11: Finetuning

* Part 11.1: Understanding Finetuning [[Video]]() [[Notebook]](t81_559_class_11_1_finetune.ipynb)
* Part 11.2: Finetuning from the Dashboard [[Video]]() [[Notebook]](t81_559_class_11_2_dashboard.ipynb)
* **Part 11.3: Finetuning from Code** [[Video]]() [[Notebook]](t81_559_class_11_3_code.ipynb)
* Part 11.4: Evaluating your Model [[Video]]() [[Notebook]](t81_559_class_11_4_eval.ipynb)
* Part 11.5: Finetuning for Text to Image [[Video]]() [[Notebook]](t81_559_class_11_5_image.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [ ]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain openai streamlit

# Part 11.3: Finetuning from Code

Just like the last part we will utilize the following training data.

* [sarcastic.jsonl](https://data.heatonresearch.com/data/t81-559/finetune/sarcastic.jsonl)
* [sarcastic_val.jsonl](https://data.heatonresearch.com/data/t81-559/finetune/sarcastic_val.jsonl)





In [ ]:
from openai import OpenAI

!wget https://data.heatonresearch.com/data/t81-559/finetune/sarcastic.jsonl
!wget https://data.heatonresearch.com/data/t81-559/finetune/sarcastic_val.jsonl

client = OpenAI()

obj = client.files.create(
  file=open("sarcastic.jsonl", "rb"),
  purpose="fine-tune"
)

In [ ]:
obj.id

In [ ]:
status

### Run and Monitor Finetuning

In [ ]:
import time
from openai import OpenAI

# Start the fine-tuning job
train = client.fine_tuning.jobs.create(
    training_file=obj.id,
    model="gpt-4o-mini-2024-07-18"
)

done = False

# Initialize a set to store processed event IDs
processed_event_ids = set()

while not done:
    # Retrieve the latest status of the fine-tuning job
    status = client.fine_tuning.jobs.retrieve(train.id)
    print(f"Job status: {status.status}")

    # Fetch all events related to the fine-tuning job
    events = client.fine_tuning.jobs.list_events(fine_tuning_job_id=train.id)

    # Collect new events that haven't been processed yet
    new_events = []
    for event in events:
        if event.id not in processed_event_ids:
            new_events.append(event)
            processed_event_ids.add(event.id)

    # Sort the new events in chronological order
    new_events.sort(key=lambda e: e.created_at)

    # Display the new events in order
    for event in new_events:
        print(f"{event.created_at}: {event.message}")

    if status.status == "succeeded":
        done = True
        print("Done!")
    elif status.status == "failed":
        done = True
        print("Failed!")
    else:
        print("Waiting for updates...")
        time.sleep(20)  # Sleep for 20 seconds


In [ ]:
model_id = status.fine_tuned_model
print(f"Trained model id: {model_id}")

### Test the Finetuned Model

In [ ]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=model_id,
  messages=[
    {"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."},
    {"role": "user", "content": "What is the capital of the USA?"}
  ]
)
print(completion.choices[0].message)

### Delete Old Models

In [ ]:
#client.models.delete("ft:gpt-4o-mini-2024-07-18:personal:sarcastic:A9yCtR0b")